Desarrollo Algoritmo para Prueba 1

1) Selección de Países y Recolección de Datos
Esta celda procesa los datos del recurso solar y condiciones meteorológicas de Chile, Sudáfrica e Indonesia.

In [2]:
import polars as pl
from pathlib import Path

# Definir los pares de archivos por ubicación
ubicaciones = {
    'Sudáfrica': {
        'lat': -29.19,
        'lon': 21.30,
        'archivos': ['1155731_-29.19_21.30_2018.csv', '1155731_-29.19_21.30_2019.csv']
    },
    'China': {
        'lat': 44.73,
        'lon': 87.66,
        'archivos': ['3480150_44.73_87.66_2018.csv', '3480150_44.73_87.66_2019.csv']
    },
    'Chile': {
        'lat': -23.84,
        'lon': -69.89,
        'archivos': ['5815755_-23.84_-69.89_2018.csv', '5815755_-23.84_-69.89_2019.csv']
    }
}

# Procesar cada ubicación
for id_ubicacion, info in ubicaciones.items():
    print(f"\nProcesando ubicación {id_ubicacion} ({info['lat']}, {info['lon']})")
    
    # Lista para almacenar los DataFrames de cada año
    dfs = []
    
    # Procesar cada archivo de la ubicación
    for archivo in info['archivos']:
        # Leer el archivo CSV, saltando las primeras 2 filas que contienen metadatos
        df = pl.read_csv(archivo, skip_rows=2)
        # Filtrar las columnas que NO contengan 'Clearsky'
        columnas_filtradas = [col for col in df.columns if 'Clearsky' not in col]
        df = df.select(columnas_filtradas)
        # Agregar columnas de ubicación
        df = df.with_columns([
            pl.lit(id_ubicacion).alias('Location_ID'),
            pl.lit(info['lat']).alias('Latitude'),
            pl.lit(info['lon']).alias('Longitude')
        ])
        
        # Agregar el DataFrame a la lista
        dfs.append(df)
    
    # Combinar los DataFrames de los dos años
    df_final = pl.concat(dfs)
    
    # Ordenar por fecha y hora
    df_final = df_final.sort(['Year', 'Month', 'Day', 'Hour', 'Minute'])
    
    # Mostrar las primeras filas del DataFrame
    print(f"\nPrimeras filas del DataFrame para ubicación {id_ubicacion}:")
    print(df_final.head())
    
    # Mostrar información básica del DataFrame
    print(f"\nInformación del DataFrame para ubicación {id_ubicacion}:")
    print(df_final.describe())
    
    # Crear directorio para los resultados si no existe
    Path('resultados').mkdir(exist_ok=True)
    
    # Guardar el DataFrame en un nuevo archivo CSV
    nombre_archivo = f'resultados/datos_{id_ubicacion}.csv'
    df_final.write_csv(nombre_archivo)
    print(f"\nLos datos han sido guardados en '{nombre_archivo}'")

print("\nProcesamiento completado para todas las ubicaciones")
print("\nPara ver el resultado completo, seleccione view as scrollable element")

#pl.Config.set_fmt_str_lengths(10000)
#pl.Config.set_tbl_rows(1000)


Procesando ubicación Sudáfrica (-29.19, 21.3)

Primeras filas del DataFrame para ubicación Sudáfrica:
shape: (5, 19)
┌──────┬───────┬─────┬──────┬───┬───────────────────┬─────────────┬──────────┬───────────┐
│ Year ┆ Month ┆ Day ┆ Hour ┆ … ┆ Relative Humidity ┆ Location_ID ┆ Latitude ┆ Longitude │
│ ---  ┆ ---   ┆ --- ┆ ---  ┆   ┆ ---               ┆ ---         ┆ ---      ┆ ---       │
│ i64  ┆ i64   ┆ i64 ┆ i64  ┆   ┆ f64               ┆ str         ┆ f64      ┆ f64       │
╞══════╪═══════╪═════╪══════╪═══╪═══════════════════╪═════════════╪══════════╪═══════════╡
│ 2018 ┆ 1     ┆ 1   ┆ 0    ┆ … ┆ 64.11             ┆ Sudáfrica   ┆ -29.19   ┆ 21.3      │
│ 2018 ┆ 1     ┆ 1   ┆ 1    ┆ … ┆ 66.25             ┆ Sudáfrica   ┆ -29.19   ┆ 21.3      │
│ 2018 ┆ 1     ┆ 1   ┆ 2    ┆ … ┆ 69.09             ┆ Sudáfrica   ┆ -29.19   ┆ 21.3      │
│ 2018 ┆ 1     ┆ 1   ┆ 3    ┆ … ┆ 70.29             ┆ Sudáfrica   ┆ -29.19   ┆ 21.3      │
│ 2018 ┆ 1     ┆ 1   ┆ 4    ┆ … ┆ 66.65             ┆ Sudáfrica

---------------------------------------------------------------------------------------------

2) Simulación de Plantas PV y CSP


Tecnología PV

In [16]:
#Simulación de producción anual de energía. 500kW, 1MW, 5MW
import PySAM
